In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import scipy.sparse
import scipy.sparse.linalg

```mermaid
graph TD
    A[게임 시작: (0, 2n)] --> B[BFS로 모든 가능한 상태 탐색]
    B --> C[각 상태에서 전이 확률 계산]
    C --> D[희소 행렬로 전이 행렬 구성]
    D --> E[기본 행렬 계산: N = (I-Q)^-1]
    E --> F[기대 턴 수 = N × 1]
    F --> G[실패 확률 분포 계산]
```

In [2]:
def remove_last_row_column(sparse_matrix):
    """목적: 마르코프 체인에서 흡수 상태(게임 완료 상태)를 제외한 Q 행렬을 만들기 위함"""
    """용도: 기대 턴 수 계산에 필요한 기본 행렬(Fundamental Matrix) 계산"""
    num_states = sparse_matrix.shape[0]
    row_mask = np.ones(sparse_matrix.shape[0], dtype=bool)
    row_mask[num_states-1] = False
    col_mask = np.ones(sparse_matrix.shape[1], dtype=bool)
    col_mask[num_states-1] = False
    return sparse_matrix[row_mask][:,col_mask]

In [3]:
def get_markov_chain(number_of_pairs, entire_matrix=False, extra_info=False):
    """마르코프 체인의 전이 확률을 계산하는 핵심"""
    # α/β: 첫 번째 카드가 이미 알고 있는 색상일 확률
    # (1-α/β) × 1/(β-1): 처음 본 색상이 운 좋게 바로 매칭될 확률
    # (1-α/β) × α/(β-1): 첫 카드는 새 색상, 두 번째는 기억 속 색상
    # (1-α/β) × (1-(α+1)/(β-1)): 두 카드 모두 새로운 서로 다른 색상
    
    queue = [(0, 2 * number_of_pairs)]
    adjacency_list = {}

    while queue:
        curr_state = queue.pop(0)  # FIFO로 변경하여 BFS 보장
        alpha, beta = curr_state
        
        if curr_state in adjacency_list:  # 이미 처리된 상태는 건너뛰기
            continue
    
        if alpha < 0 or beta < 0:
            continue
  
        next_states = {}
        if alpha > beta or (alpha + beta) % 2 == 1: # Know more knowns than unknowns or there is a duplicate in the flipped cards
            next_states[(alpha - 1, beta)] = 1
        elif alpha == 0 and beta == 0:
            next_states[(0, 0)] = 1
        elif alpha == beta:
            next_states[(alpha - 1, beta - 1)] = 1
        else:
            next_states[(alpha - 1, beta - 1)] = alpha / beta
            next_states[(alpha + 2, beta - 2)] = (1 - alpha/beta) * (1 - (alpha+1)/(beta-1))
            next_states[(alpha + 1, beta - 2)] = (1 - alpha/beta) * (alpha / (beta - 1))
            next_states[(alpha, beta - 2)]     = (1 - alpha/beta) * 1 / (beta - 1)

        # 유효한 상태만 추가 (음수나 불가능한 상태 제외)
        valid_next_states = {}
        for state, prob in next_states.items():
            if prob > 0 and state[0] >= 0 and state[1] >= 0:
                valid_next_states[state] = prob
                if state not in adjacency_list:  # 아직 처리되지 않은 상태만 큐에 추가
                    queue.append(state)

        adjacency_list[curr_state] = valid_next_states

    # 모든 상태가 수집된 후 마르코프 체인 행렬 구성
    num_states = len(adjacency_list)
    markov_chain = scipy.sparse.lil_matrix((num_states, num_states))

    # Build enumeration for states
    sorted_keys = sorted(list(adjacency_list.keys()), key=lambda pair: -sum(pair))
    keys_to_index = {key: index for index, key in enumerate(sorted_keys)}

    for state, neighbors in adjacency_list.items():
        for destination, probability in neighbors.items():
            if destination in keys_to_index:  # 목적지 상태가 존재하는지 확인
                markov_chain[(keys_to_index[state], keys_to_index[destination])] = probability
    
    if not entire_matrix: # Get block matrix for fundamental matrix calculation
        to_return = remove_last_row_column(markov_chain)
    else:
        to_return = scipy.sparse.csr_matrix(markov_chain)

    if extra_info:
        return to_return, { "adjacency_list": adjacency_list, "keys_to_index": keys_to_index, "sorted_keys": sorted_keys }
    return to_return

In [4]:
def get_case_info(n_pairs):
    """기대 턴 수 계산: (I - Q)^(-1) × 1"""
    # 흡수 마르코프 체인의 핵심 공식을 사용:
        # 기본 행렬: N = (I - Q)^(-1)
        # 기대 흡수 시간: N × 1
        
    markov_matrix = get_markov_chain(n_pairs, entire_matrix=True)
    Q_matrix = remove_last_row_column(markov_matrix)

    # Solve with the fundamental matrix
    expected_number_of_turns = scipy.sparse.linalg.spsolve( scipy.sparse.eye(Q_matrix.shape[0]) - Q_matrix, np.ones(Q_matrix.shape[0]))[0]
    expected_number_of_failures = expected_number_of_turns - n_pairs

    starting_point = np.zeros(markov_matrix.shape[0])
    starting_point[0] = 1

    first_chance_to_win = markov_matrix.T ** n_pairs
    prob_vector = first_chance_to_win @ starting_point
    chance_to_finish_with_failures = [prob_vector[-1]]

    # Almost all the processing time is calculating the failure distribution
    for number_of_failures in range(n_pairs):
        next_prob_vector = markov_matrix.T @ prob_vector
        chance_to_finish_with_failures.append(next_prob_vector[-1] - prob_vector[-1])
        prob_vector = next_prob_vector
    
    return np.array(chance_to_finish_with_failures)




In [5]:
pd.DataFrame(get_case_info(7))

,0
0,0.000007
1,0.000829
2,0.027972
3,0.283450
4,0.567521
5,0.119288
6,0.000932
7,0.000000


In [6]:
get_markov_chain(3, entire_matrix=True)

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>